## setup

In [1]:
import requests
from pprint import pprint,pformat

elucidate_base_url = 'http://localhost:8080/annotation'
headers = {
    'Accept':'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"',
    'Content-Type': 'application/ld+json; profile="http://www.w3.org/ns/anno.jsonld"'
}

def get_etag(url:str)->str:
    return requests.get(url).headers['etag'][3:-1]

## create container

In [7]:
body = {
  "@context": [
    "http://www.w3.org/ns/anno.jsonld",
    "http://www.w3.org/ns/ldp.jsonld"
  ],
  "type": [
    "BasicContainer",
    "AnnotationCollection"
  ],
  "label": "A Container for Web Annotations"
}

response = requests.post(url=elucidate_base_url+"/w3c/",headers=headers,json=body)
container_uri = response.json()['id']

response.status_code
url =response.headers['location']
url.split('/')[-2]

'e7fd56f6-7dc8-442a-b18a-a9b4a71f36f1'

## read container

In [3]:
requests.get(container_uri).json()

{'@context': ['http://www.w3.org/ns/anno.jsonld',
  'http://www.w3.org/ns/ldp.jsonld'],
 'id': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/',
 'type': 'AnnotationCollection',
 'label': 'A Container for Web Annotations',
 'first': {'type': 'AnnotationPage',
  'as:items': {'@list': []},
  'partOf': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/',
  'startIndex': 0},
 'last': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/?page=0&desc=1',
 'total': 0}

## create annotation

In [4]:
annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "type": "Annotation",
  "body": {
    "type": "TextualBody",
    "value": "I like this page!"
  },
  "target": "http://www.example.com/index.html"
}

response = requests.post(
    url=container_uri,
    headers=headers,
    json=annotation)
annotation_uri = response.json()['id']

response.status_code

201

response headers:

In [5]:
for (k,v) in sorted(response.headers.items()):
    print(f"{k} = {v}")

Allow = PUT,GET,OPTIONS,HEAD,DELETE
Cache-Control = no-cache, no-store, max-age=0, must-revalidate
Content-Type = application/ld+json; profile="http://www.w3.org/ns/anno.jsonld";charset=UTF-8
Date = Thu, 08 Apr 2021 14:15:16 GMT
ETag = W/"985759a998e93664698ced20228fda82"
Expires = 0
Link = <http://www.w3.org/ns/ldp#Resource>; rel="type"
Location = http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6
Memento-Datetime = Thu, 08 Apr 2021 14:15:16 UTC
Pragma = no-cache
Transfer-Encoding = chunked
Vary = Origin, Access-Control-Request-Method, Access-Control-Request-Headers, Accept
X-Content-Type-Options = nosniff
X-Frame-Options = DENY
X-XSS-Protection = 1; mode=block


response body:

In [6]:
response.json()

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6',
 'type': 'Annotation',
 'body': {'type': 'TextualBody', 'value': 'I like this page!'},
 'target': 'http://www.example.com/index.html'}

## read annotation

- W3C :

In [7]:
pprint(annotation_uri)
requests.get(annotation_uri).json()

'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6'


{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6',
 'type': 'Annotation',
 'body': {'type': 'TextualBody', 'value': 'I like this page!'},
 'target': 'http://www.example.com/index.html'}

- OA :

In [8]:
oa_uri = annotation_uri.replace('/w3c/','/oa/')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6',
 '@type': 'oa:Annotation',
 'hasBody': {'@type': 'oa:TextualBody', 'value': 'I like this page!'},
 'hasTarget': 'http://www.example.com/index.html'}

## update annotation

In [9]:
etag = get_etag(annotation_uri)
new_annotation = {
  "@context": "http://www.w3.org/ns/anno.jsonld",
  "id": "http://example.org/w3c/mycontainer/anno1",
  "type": "Annotation",
  "created": "2015-01-31T12:03:45Z",
  "body": {
    "type": "TextualBody",
    "value": "I don't like this page!"
  },
  "target": "http://www.example.com/index.html"
}
update_headers = {'If-Match':etag}
update_headers.update(headers)
response = requests.put(url=annotation_uri,headers=update_headers,json=new_annotation)

response.status_code

200

In [10]:
requests.get(annotation_uri).json()

{'@context': 'http://www.w3.org/ns/anno.jsonld',
 'id': 'http://localhost:8080/annotation/w3c/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6',
 'type': 'Annotation',
 'created': '2015-01-31T12:03:45Z',
 'body': {'type': 'TextualBody', 'value': "I don't like this page!"},
 'target': 'http://www.example.com/index.html'}

In [11]:
oa_uri = annotation_uri.replace('w3c','oa')
pprint(oa_uri)
requests.get(oa_uri).json()

'http://localhost:8080/annotation/oa/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6'


{'@context': 'https://raw.githubusercontent.com/dlcs/elucidate-server/master/elucidate-server/src/main/resources/contexts/oa.jsonld',
 '@id': 'http://localhost:8080/annotation/oa/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6',
 '@type': 'oa:Annotation',
 'annotatedAt': '2015-01-31T12:03:45Z',
 'hasBody': {'@type': 'oa:TextualBody', 'value': "I don't like this page!"},
 'hasTarget': 'http://www.example.com/index.html'}

## annotation history

In [12]:
response = requests.get(annotation_uri)
for (k,v) in sorted(response.headers.items()):
    print(f"{k} = {v}")

Allow = PUT,GET,OPTIONS,HEAD,DELETE
Cache-Control = no-cache, no-store, max-age=0, must-revalidate
Content-Type = application/ld+json;charset=UTF-8
Date = Thu, 08 Apr 2021 14:15:55 GMT
ETag = W/"4389921286a3eba7230ad281c34e896c"
Expires = 0
Link = <http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6/1>; rel="prev memento"; datetime="Thu, 08 Apr 2021 14:15:16 UTC"
Memento-Datetime = Thu, 08 Apr 2021 14:15:42 UTC
Pragma = no-cache
Transfer-Encoding = chunked
Vary = Origin, Access-Control-Request-Method, Access-Control-Request-Headers, Accept
X-Content-Type-Options = nosniff
X-Frame-Options = DENY
X-XSS-Protection = 1; mode=block


In [13]:
response.headers['link']

'<http://www.w3.org/ns/ldp#Resource>; rel="type", <http://localhost:8080/annotation/w3c/services/history/162b641a-04c7-43d1-8c98-c4e1a103b4fb/1ecc0f23-523d-4233-82f6-4a083c2f73a6/1>; rel="prev memento"; datetime="Thu, 08 Apr 2021 14:15:16 UTC"'

## delete annotation

In [14]:
etag = get_etag(annotation_uri)
delete_headers = {'If-Match':etag}
delete_headers.update(headers)
response = requests.delete(url=annotation_uri,headers=delete_headers)

response.status_code

204

In [15]:
requests.get(annotation_uri).status_code

410

In [16]:
response = requests.get("http://localhost:8080/annotation/w3c/services/history/9c153658-6748-4fbe-bd41-c72554e67571/d9dcece4-04a7-4b70-873f-f66cb9f3730e/1")
response.status_code

404